<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/wordmark_black65ee464.png" width="700">

# Week 2 : Final Challenge

**Welcome to the final challenge!**  

In the previous notebook we've seen how we can use the VQC class in Aqua to classify the digits `0` and `1`. However, classifying `0` and `1` is relatively simple as digits `0` and `1` are easily distinguishable. `4` and `9` however, are notoriously similar, with a _loop_ on the top and a _line_ on the bottom. This can be corroborated looking at our 2-D t-SNE plot from the previous notebook (Fig.2), we see that `0` and `1` are clustered relatively far from each other making them easily distinguishable, however `4` and `9` are overlapping. In this challenge we are providing you with a dataset with digits reduced to **dimension 3**. For example, in Fig.1 we can see the dimension reduction of the 784 dimension vector for digit `4` into a dimension 3 feature vector. 

**Fig.1 : Features of the digit `4` after reducing dimension to 3:** 
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/four2a7701f.png" width="700">

**Fig.2 : MNIST dataset after dimension reduction to 2 as given in the previous notebook:**
<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/mnist_plot53adb39.png" width="400">

## Challenge Question   
Use the VQC method from Aqua to classify the digits `4` and `9` as given in the dataset **challenge_dataset_4_9_dim3_training.csv** provided to you. 

## Rules and Guidelines

* Your `QuantumCircuit` can have a **maximum of 6 qubits**.
* **Cost of the circuit should be less than 2000**.  
* You should not change names of the functions `feature_map()` , `variational_circuit()`  and `return_optimal_params()`.
* All the functions must return the value types as mentioned. 
* All circuits must be Qiskit generated.
* Best of all submissions is considered for grading.

## Judging criteria 

* Primary judgement is based on the **accuracy of the model**, higher the better. **Accuracies which differ by less than 0.005 will be considered to be equal**. ex: Accuracies 0.7783 and 0.7741 will be considered to be equal.
* If the accuracies are tied, the tie will be broken using **cost of the circuit** as the metric, lower the better. 
* In the case that both accuracy of the model and cost of the circuit are equal, **time of submission** is taken into account, Earlier the better. 

_**Important Note:**_ The **leaderboard shown during the progress of the competition** will only display accuracy of the model and is **not the final leaderboard**. Breaking ties between accuracy of the model by considering lower **cost of circuit** will only be done after the competition ends. **The final leaderboard will be announced post the event** which will take into consideration cost of the circuit and time of submission. 

## Certificate Eligibility

Everyone who scores an **accuracy greater than 0.70 (i.e, 70%) will be eligible for a certificate**. 


An explanation on how to calculate the accuracy of the model and the cost of the circuit is given in the end inside the `grade()` function. Before you submit, make sure the grading function is running on your device. To save time you can also use the grading function provided to calculate the accuracy and circuit cost without having to submit your solution onto HackerEarth. Remember, your final score will be determined using the same grading methods as given in this notebook, but will be evaluated on unseen datapoints.

In [1]:
# installing a few dependencies
!pip install --upgrade seaborn==0.10.1
!pip install --upgrade scikit-learn==0.23.1
!pip install --upgrade matplotlib==3.2.0
!pip install --upgrade pandas==1.0.4
!pip install --upgrade qiskit==0.19.6 
!pip install --upgrade plotly==4.9.0

# the output will be cleared after installation
from IPython.display import clear_output
clear_output()

In [68]:
# we have imported a few libraries we thing might be useful 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer

from qiskit import *
import numpy as np
from qiskit.visualization import plot_bloch_multivector, plot_histogram
%matplotlib inline
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap, RealAmplitudes, EfficientSU2
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.optimizers import COBYLA,ADAM, SPSA, SLSQP


# The the write_and_run() magic function creates a file with the content inside the cell that it is run. 
# You have used this in previous exercises for creating your submission files. 
# It will be used for the same purpose here.

from IPython.core.magic import register_cell_magic
@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

# Solution

## Data loading 

This notebook has helper functions and code snippets to save your time and help you concentrate on what's important: Increasing the accuracy of your model. Running the cell below will import the challenge dataset and will be available to you as `data`. Before running the cell below store the dataset in this file structure (or change the `data_path` accordingly):  

- `challenge_notebook.ipynb`
- `dataset`
    - `challenge_dataset_4_9.csv`


In [36]:
data_path=''
data = np.loadtxt(data_path + "challenge_dataset_4_9_dim3_training.csv", delimiter=",")

# extracting the first column which contains the labels
data_labels = data[:, :1].reshape(data.shape[0],)
# extracting all the columns but the first which are our features
data_features = data[:, 1:]

## Visualizing the dataset

Before we dive into solving the question it is always beneficial to look at the dataset pictographically. This will help us understand patterns which we could leverage when designing our feature maps and variational circuits for example.

In [37]:
import plotly.express as px
import pandas as pd

# creating a dataframe using pandas only for the purpose fo plotting
df = pd.DataFrame({'Component 0':data_features[:,0], 'Component 1':data_features[:,1], 
                   'Component 2':data_features[:,2], 'label':data_labels})

fig = px.scatter_3d(df, x='Component 0', y='Component 1', z='Component 2', color='label')
fig.show()

## Extracting the training dataset

The given dataset has already been reduced in dimension and normalized, so, further pre-processing isn't techincally required. You can do so if you want to, but the testing dataset will be of the same dimension and normalisation as the training dataset provided. Training a dataset of size 6,000 will take multiple hours so you'll need to extract a subset of the dataset to use as a training dataset. The accuracy of the model may vary based on the datapoints and size of the training dataset you choose. Thus, experimenting with various sizes and datapoints will be necessary. For example, Increasing the training dataset size may increase the accuracy of the model however it will increase the training time as well.

Use the space below to extract your training dataset from `data`. For your convenience `data` has been segregated into `data_labels` and `data_features`.

* `data_labels` : 6,000 $\times$ 1 column vector with each entry either `4` or `9` 
* `data_features` : 6,000 $\times$ 3 matrix with each row having the feature corresponding to the label in `data_labels`

**Note:** This process was done in the previous [VQC notebook](https://github.com/Qiskit-Challenge-India/2020/blob/master/Day%206%2C%207%2C8/VQC_notebook.ipynb) with `0` and `1` labels and can be modified and used here as well. 

In [38]:
### WRITE YOUR CODE BETWEEN THESE LINES - START

# do your classical pre-processing here

four_datapoints = []
nine_datapoints = []

for i in range(6000):
    if data_labels[i] == 4:                   # extracting 4s
        four_datapoints.append(data_features[i])

for i in range(6000):
    if data_labels[i] == 9:                   # extracting 9s
        nine_datapoints.append(data_features[i])
        
four_datapoints = np.array(four_datapoints)
nine_datapoints = np.array(nine_datapoints)

train_size = 20
test_size = 10

four_train = four_datapoints[:train_size]
nine_train = nine_datapoints[:train_size]

four_test = four_datapoints[train_size + 1:train_size + test_size + 1]
nine_test = nine_datapoints[train_size + 1:train_size + test_size + 1]

# store your training and testing datasets to be input in the VQC optimizer in the "training_input" and 
# "testing_input" variables respectively. These variables will be accessed while creating a VQC instance later. 
training_input = {'A':four_train, 'B':nine_train}
test_input = {'A':four_test, 'B':nine_test}

### WRITE YOUR CODE BETWEEN THESE LINES - END

## Building a Quantum Feature Map

Given below is the `feature_map()` function. It takes no input and has to return a feature map which is either a `FeatureMap` or `QuantumCircuit` object. In the previous notebook you've learnt how feature maps work and the process of using existing feature maps in Qiskit or creating your own. In the space given **inside the function** you have to create a feature map and return it.   


**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `feature_map.py`

In [102]:
%%write_and_run feature_map.py
# the write_and_run function writes the content in this cell into the file "feature_map.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZZFeatureMap, ZFeatureMap, PauliFeatureMap
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def feature_map(): 
    # BUILD FEATURE MAP HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
    # build the feature map
    feature_map = ZZFeatureMap(feature_dimension=3, reps=3, entanglement='full')
    
    # BUILD FEATURE MAP HERE - END
    
    #return the feature map which is either a FeatureMap or QuantumCircuit object
    return feature_map

## Building a Variational Circuit

Given below is the `variational_circuit()` function. It takes no input and has to return a variational circuit which is either a `VariationalForm` or `QuantumCircuit` object. In the previous notebook you've learnt how variational circuits work and the process of using existing variational circuit in Qiskit or creating your own. You have to create a variational circuit in the space given **inside the function** and return it. You can find various variational circuits in the [Qiskit Circuit Library](https://qiskit.org/documentation/apidoc/circuit_library.html) under N-local circuits.

**IMPORTANT:** 
* If you require Qiskit import statements other than the ones provided in the cell below, please include them inside the appropriate space provided. **All additional import statements must be Qiskit imports.** 
* the first line of the cell below must be `%%write_and_run feature_map.py`. This function stores the content of the cell below in the file `variational_circuit.py`

In [103]:
%%write_and_run variational_circuit.py
# the write_and_run function writes the content in this cell into the file "variational_circuit.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import  RealAmplitudes, EfficientSU2
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def variational_circuit():
    # BUILD VARIATIONAL CIRCUIT HERE - START
    
    # import required qiskit libraries if additional libraries are required
    
    # build the variational circuit
    var_circuit = EfficientSU2(3, reps=2)

    # BUILD VARIATIONAL CIRCUIT HERE - END
    
    # return the variational circuit which is either a VaritionalForm or QuantumCircuit object
    return var_circuit

## Choosing a Classical Optimizer

In the `classical_optimizer()` function given below you will have to import the optimizer of your choice from [`qiskit.aqua.optimizers`](https://qiskit.org/documentation/apidoc/qiskit.aqua.components.optimizers.html) and return it. This function will not be called by the grading function `grade()` and thus the name of the function `classical_optimizer()`can be changed if needed. 

In [104]:
def classical_optimizer():
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - START
    
    # import the required clasical optimizer from qiskit.aqua.optimizers
    
    # create an optimizer object
    cls_opt = SPSA(500)

    
    # CHOOSE AND RETURN CLASSICAL OPTIMIZER OBJECT - END
    return cls_opt

### Callback Function

The `VQC` class can take in a callback function to which the following parameters will be passed after every optimization cycle of the algorithm:

* `eval_count` : the evaulation counter
* `var_params` : value of parameters of the variational circuit
* `eval_val`  : current cross entropy cost 
* `index` : the batch index

In [105]:
def call_back_vqc(eval_count, var_params, eval_val, index):
    print("eval_count: {}".format(eval_count))
    print("var_params: {}".format(var_params))
    print("eval_val: {}".format(eval_val))
    print("index: {}".format(index))

## Optimization Step

This is where the whole VQC algorithm will come together. First we create an instance of the `VQC` class. 

In [106]:
# a fixed seed so that we get the same answer when the same input is given. 
seed = 10598

# setting our backend to qasm_simulator with the "statevector" method on. This particular setup is given as it was 
# found to perform better than most. Feel free to play around with different backend options.
backend = Aer.get_backend('qasm_simulator')
backend_options = {"method": "statevector"}

# creating a quantum instance using the backend and backend options taken before
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, 
                                   backend_options=backend_options)

# creating a VQC instance which you will be used for training. Make sure you input the correct training_dataset and 
# testing_dataset as defined in your program.
vqc = VQC(optimizer=classical_optimizer(), 
          feature_map=feature_map(), 
          var_form=variational_circuit(), 
          callback=call_back_vqc, 
          training_dataset=training_input,     # training_input must be initialized with your training dataset
          test_dataset=test_input)             # testing_input must be initialized with your testing dataset

Now, let's run the VQC classification routine

In [107]:
start = time.process_time()

result = vqc.run(quantum_instance)

print("time taken: ")
print(time.process_time() - start)

print("testing success ratio: {}".format(result['testing_accuracy']))

eval_count: 0
var_params: [-0.03314996  0.45871601 -0.04825141  1.09618094 -0.40505076  1.27167183
 -0.26798164  1.22763485 -0.84761893 -1.30956245  0.03313513  0.17226306
 -1.99823296 -0.36402171  0.18341837  0.65470307 -0.90378223  0.41497065]
eval_val: 0.923085318250474
index: 0
eval_count: 1
var_params: [ 0.16685004  0.65871601  0.15174859  1.29618094 -0.20505076  1.07167183
 -0.46798164  1.02763485 -0.64761893 -1.10956245 -0.16686487 -0.02773694
 -2.19823296 -0.56402171  0.38341837  0.85470307 -1.10378223  0.21497065]
eval_val: 0.9785085266984057
index: 1
eval_count: 2
var_params: [-0.03314996  0.45871601 -0.04825141  1.09618094 -0.20505076  1.07167183
 -0.46798164  1.02763485 -0.84761893 -1.30956245  0.03313513 -0.02773694
 -2.19823296 -0.56402171  0.18341837  0.65470307 -1.10378223  0.21497065]
eval_val: 0.9390751181313387
index: 2
eval_count: 3
var_params: [ 0.16685004  0.65871601  0.15174859  1.29618094 -0.40505076  1.27167183
 -0.26798164  1.22763485 -0.64761893 -1.10956245 -

eval_count: 29
var_params: [-0.03314996  0.45871601  0.15174859  1.09618094 -0.40505076  1.27167183
 -0.46798164  1.02763485 -0.64761893 -1.10956245  0.03313513 -0.02773694
 -2.19823296 -0.56402171  0.18341837  0.85470307 -1.10378223  0.41497065]
eval_val: 0.9287367109991926
index: 29
eval_count: 30
var_params: [ 0.16685004  0.45871601  0.15174859  1.29618094 -0.20505076  1.07167183
 -0.26798164  1.22763485 -0.84761893 -1.10956245  0.03313513 -0.02773694
 -1.99823296 -0.36402171  0.38341837  0.65470307 -1.10378223  0.21497065]
eval_val: 0.9716855662157691
index: 30
eval_count: 31
var_params: [-0.03314996  0.65871601 -0.04825141  1.09618094 -0.40505076  1.27167183
 -0.46798164  1.02763485 -0.64761893 -1.30956245 -0.16686487  0.17226306
 -2.19823296 -0.56402171  0.18341837  0.85470307 -0.90378223  0.41497065]
eval_val: 0.9183594353390795
index: 31
eval_count: 32
var_params: [-0.03314996  0.45871601  0.15174859  1.29618094 -0.20505076  1.27167183
 -0.46798164  1.02763485 -0.84761893 -1.30

eval_count: 58
var_params: [ 1.08932937  0.90781083 -0.97073073  1.52573535 -0.82413975  1.47545476
 -0.03842723  1.2418832  -0.63337058 -2.25158218 -0.56641345 -0.44682593
 -2.50691535  0.57799803  0.80250736  0.9933913  -0.48469324  0.09582283]
eval_val: 0.714268137505587
index: 58
eval_count: 59
var_params: [ 0.9193352   1.077805   -0.80073656  1.69572952 -0.65414558  1.30546059
  0.13156694  1.41187737 -0.46337641 -2.08158801 -0.39641928 -0.27683176
 -2.33692118  0.40800386  0.63251319  1.16338547 -0.65468741 -0.07417134]
eval_val: 0.7184069309612329
index: 59
eval_count: 60
var_params: [ 1.1030339   0.89410629 -0.98443527  1.67892328 -0.67095182  1.48915929
  0.1147607   1.22817866 -0.64707512 -2.26528672 -0.58011799 -0.46053046
 -2.52061989  0.59170256  0.81621189  0.97968676 -0.4709887  -0.0573651 ]
eval_val: 0.7223961443274451
index: 60
eval_count: 61
var_params: [ 0.93614144  1.06099875 -0.81754281  1.51203082 -0.83784429  1.32226683
 -0.05213176  1.39507112 -0.48018265 -2.098

eval_count: 87
var_params: [ 0.95310685  0.68999797 -1.12551666  1.63550649 -1.01519221  0.92312322
  0.20494666  1.08985663 -0.41866929 -1.94099172 -0.31553637 -0.79678476
 -2.41887056  0.98138656  1.03360641  1.213788   -0.7046059   0.44313475]
eval_val: 0.6568482997805697
index: 87
eval_count: 88
var_params: [ 1.11126327  0.53184156 -1.13588957  1.7936629  -1.17334863  0.93349613
  0.04679025  1.07948373 -0.4290422  -1.95136463 -0.30516346 -0.78641185
 -2.57702697  0.82323015  1.02323351  1.2241609  -0.54644949  0.43276184]
eval_val: 0.6513556036226265
index: 88
eval_count: 89
var_params: [ 0.96347976  0.67962507 -1.28367308  1.6458794  -1.02556512  1.08127963
  0.19457375  0.93170022 -0.5768257  -2.09914814 -0.15737995 -0.63862835
 -2.42924347  0.97101366  0.87545     1.37194441 -0.69423299  0.28497833]
eval_val: 0.6492353240943263
index: 89
eval_count: 90
var_params: [ 0.95994078  0.68316405 -1.28721206  1.78939747 -1.02202614  0.93776156
  0.19811273  1.07521829 -0.58036468 -2.10

eval_count: 116
var_params: [ 0.96288186  0.29108435 -1.25562068  1.52020287 -1.27192044  1.0045325
  0.2433846   1.03690744 -0.57149372 -2.05444966 -0.384613   -0.90382859
 -2.47379914  0.88417336  1.14090372  1.40983538 -0.89012781  0.27581227]
eval_val: 0.5895254672279655
index: 116
eval_count: 117
var_params: [ 1.10295363  0.43115613 -1.39569245  1.66027464 -1.41199221  0.86446072
  0.10331283  0.89683567 -0.71156549 -1.91437789 -0.52468477 -0.76375681
 -2.33372737  0.74410159  1.00083194  1.26976361 -0.75005604  0.1357405 ]
eval_val: 0.5904979143940616
index: 117
eval_count: 118
var_params: [ 1.10137697  0.42957946 -1.2544535   1.65869797 -1.41041554  0.86603739
  0.10488949  0.89841234 -0.57032654 -1.91595456 -0.5231081  -0.90499576
 -2.47496631  0.88534053  1.00240861  1.41100255 -0.75163271  0.27697944]
eval_val: 0.5945714940708099
index: 118
eval_count: 119
var_params: [ 0.96171469  0.28991718 -1.39411578  1.51903569 -1.27075326  1.00569967
  0.24455177  1.03807462 -0.70998882

...

eval_count: 933
var_params: [ 1.50879295 -0.24740407 -1.67654154  1.80849493 -1.89171926  0.56120049
  0.03746424  0.8716213  -0.56531332 -2.9673369  -0.23505119 -1.12693154
 -3.0460307   0.79209748  1.38392175  1.50507606 -1.39622291 -0.0242322 ]
eval_val: 0.447039047735602
index: 933
eval_count: 934
var_params: [ 1.50831461 -0.13884626 -1.67701988  1.69993712 -1.8921976   0.45264268
  0.14602205  0.76306348 -0.56483498 -2.96685856 -0.23552953 -1.01837372
 -3.15458851  0.68353966  1.27536394  1.5055544  -1.39670125 -0.13279001]
eval_val: 0.4464632326953894
index: 934
eval_count: 935
var_params: [ 1.40023514 -0.24692573 -1.78509936  1.80801659 -2.00027707  0.56072215
  0.03794258  0.87114296 -0.4567555  -2.85877909 -0.343609   -1.1264532
 -3.04650904  0.79161914  1.38344341  1.61363387 -1.50478073 -0.02471054]
eval_val: 0.4464834120522241
index: 935
eval_count: 936
var_params: [ 1.40025531 -0.24690556 -1.67702432  1.69994156 -2.0002569   0.56070198
  0.03796275  0.76306792 -0.45677568 

eval_count: 962
var_params: [ 1.3981148  -0.25101798 -1.7878623   1.70016995 -1.88997922  0.55979821
  0.04146764  0.76573154 -0.4580676  -2.96173718 -0.23197833 -1.12727681
 -3.05386605  0.7903276   1.38350668  1.50104298 -1.50575819 -0.13381267]
eval_val: 0.44545664413228253
index: 962
eval_count: 963
var_params: [ 1.50585517 -0.14327761 -1.68012193  1.80791032 -1.99771959  0.45205784
  0.149208    0.8734719  -0.56580796 -2.85399681 -0.3397187  -1.01953644
 -3.16160642  0.68258723  1.27576631  1.60878335 -1.39801782 -0.0260723 ]
eval_val: 0.4496380161802561
index: 963
eval_count: 964
var_params: [ 1.39652184 -0.25261094 -1.78945525  1.69857699 -1.99610285  0.56139116
  0.14759126  0.76413858 -0.56419122 -2.85561355 -0.33810196 -1.12886977
 -3.15998968  0.68420397  1.38509964  1.60716661 -1.39963456 -0.02768904]
eval_val: 0.44698749291742346
index: 964
eval_count: 965
var_params: [ 1.50423843 -0.14489435 -1.68173867  1.80629358 -1.88838626  0.45367458
  0.03987468  0.87185516 -0.45647

eval_count: 991
var_params: [ 1.51197581 -0.14433254 -1.68005171  1.81197746 -1.89057271  0.4481927
  0.1446919   0.8754832  -0.45657498 -2.86015765 -0.22910469 -1.0246333
 -3.05694286  0.68780257  1.37760322  1.50267348 -1.51083727 -0.13047703]
eval_val: 0.44830374369252884
index: 991
eval_count: 992
var_params: [ 1.40360848 -0.25269987 -1.78841904  1.81099964 -1.89155053  0.55656002
  0.14371408  0.87450539 -0.56494231 -2.96852498 -0.23008251 -1.13300062
 -3.16531019  0.68878039  1.26923589  1.6110408  -1.50985946 -0.02210971]
eval_val: 0.443374368236074
index: 992
eval_count: 993
var_params: [ 1.51099799 -0.14531036 -1.68102953  1.70361013 -1.99894004  0.44917052
  0.03632458  0.76711588 -0.4575528  -2.86113547 -0.33747202 -1.02561111
 -3.05792068  0.7961699   1.3766254   1.5036513  -1.40246995 -0.12949922]
eval_val: 0.44849680917923934
index: 993
eval_count: 994
var_params: [ 1.50905203 -0.25462288 -1.68297549  1.7055561  -1.99699408  0.55848303
  0.14563709  0.76906184 -0.45949876

eval_count: 1020
var_params: [ 1.40708393 -0.14986335 -1.68601661  1.70404584 -1.99762188  0.55244922
  0.03582695  0.77237519 -0.56488489 -2.86422026 -0.33835275 -1.0307671
 -3.06539633  0.79524433  1.3748975   1.50297021 -1.5136969  -0.13169327]
eval_val: 0.44878703269491826
index: 1020
eval_count: 1021
var_params: [ 1.51415687 -0.25693629 -1.79308955  1.81111878 -1.89054894  0.44537628
  0.14289989  0.87944813 -0.45781195 -2.9712932  -0.23127981 -1.13784004
 -3.17246928  0.68817139  1.26782456  1.61004315 -1.40662395 -0.02462033]
eval_val: 0.44329788168063305
index: 1021
eval_count: 1022
var_params: [ 1.5161886  -0.25896802 -1.68807056  1.70609979 -1.88851721  0.55039527
  0.0378809   0.88147985 -0.45578022 -2.97332493 -0.22924808 -1.03282105
 -3.06745029  0.79319038  1.37284355  1.61207488 -1.40459223 -0.0225886 ]
eval_val: 0.44487144964088127
index: 1022
eval_count: 1023
var_params: [ 1.40913788 -0.1519173  -1.79512127  1.81315051 -1.99556793  0.44334455
  0.14493161  0.77442914 -

eval_count: 1049
var_params: [ 1.40786327 -0.1558053  -1.79676884  1.8161589  -1.99785367  0.43952444
  0.0410731   0.77029284 -0.45603293 -2.8603893  -0.3328606  -1.03617382
 -3.07755709  0.78718806  1.36757101  1.5093077  -1.41202247 -0.02673676]
eval_val: 0.4470501639166347
index: 1049
eval_count: 1050
var_params: [ 1.46263116 -0.21057319 -1.74200095  1.761391   -1.94308578  0.49429233
  0.09584099  0.82506074 -0.51080083 -2.91515719 -0.27809271 -1.09094171
 -3.13232498  0.73242016  1.31280312  1.56407559 -1.46679036 -0.08150465]
eval_val: 0.442544093175953
index: 1050
time taken: 
1009.2077086790002
testing success ratio: 0.75


## Storing the optimal parameters for grading

Once the training step of the vqc algorithm is done we obtain the optimal parameters for our specific variational form. For the grading function to be able to access these optimal parameters you will need to follow the steps below. 

* **Step 1**: Run the cell below with `print(repr(vqc.optimal_params))`. 
* **Step 2**: Copy the matrix of optimal parameters and store it in the variable `optimal_parameters` inside the function `return_optimal_params()` in the next cell. This will enable us to extract it while calculating the accuracy your the model during grading. Given below is a pictographical explanation of the same:  

<img src="https://s3-ap-southeast-1.amazonaws.com/he-public-data/opt_params456b075.png" width="800">

In [109]:
print(repr(vqc.optimal_params))

array([ 1.46263116, -0.21057319, -1.74200095,  1.761391  , -1.94308578,
        0.49429233,  0.09584099,  0.82506074, -0.51080083, -2.91515719,
       -0.27809271, -1.09094171, -3.13232498,  0.73242016,  1.31280312,
        1.56407559, -1.46679036, -0.08150465])


In [110]:
%%write_and_run optimal_params.py
# # the write_and_run function writes the content in this cell into the file "optimal_params.py"

### WRITE YOUR CODE BETWEEN THESE LINES - START
    
# import libraries that are used in the function below.
import numpy as np
    
### WRITE YOUR CODE BETWEEN THESE LINES - END

def return_optimal_params():
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    
    optimal_parameters = [ 1.46263116, -0.21057319, -1.74200095,  1.761391  , -1.94308578,
        0.49429233,  0.09584099,  0.82506074, -0.51080083, -2.91515719,
       -0.27809271, -1.09094171, -3.13232498,  0.73242016,  1.31280312,
        1.56407559, -1.46679036, -0.08150465]
    
    # STORE THE OPTIMAL PARAMETERS AS AN ARRAY IN THE VARIABLE optimal_parameters 
    return np.array(optimal_parameters)

## Submission

Before we go any further, check that you have the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` in the **same working directory as this notebook**. If you do not, then go back to the start and run the notebook making sure you have filled in the code where its required. When you run the cell below, all the three files `feature_map.py`, `variational_circuit.py` and `optimal_params.py` are combined into one file named **"answer.py"**. Now your working directory will have four python (.py) files out of which **"answer.py"** is the submission file: 
* `answer.py` <- upload this file onto HackerEarth and click on "Submit and Evaluate"
* `feature_map.py`
* `variational_circuit.py`
* `optimal_params.py`

In [111]:
solution = ['feature_map.py','variational_circuit.py','optimal_params.py']
file = open("answer.py","w")
file.truncate(0)
for i in solution:    
    with open(i) as f:
        with open("answer.py", "a") as f1:
            for line in f:
                f1.write(line)
file.close()

## Grading Function

Given below is the grading function that we shall use to grade your submission with a test dataset that is of the same format as `challenge_dataset_4_9.csv`. You can use it to grade your submission by extracting a few points out of the `challenge_dataset_4_9.csv` to get a basic idea of how your model is performing. 

In [112]:
#imports required for the grading function 
from qiskit import *
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQC
from qiskit.aqua.components.feature_maps import FeatureMap
from qiskit.aqua.components.variational_forms import VariationalForm
import numpy as np

### Working of the grading function

The grading function `grade()` takes as **input**: 

* `test_data`: (`np.ndarray`) -- **no. of datapoints $\times$ dimension of data** : the datapoints against which we want to test our model. 


* `test_labels`: (`np.ndarray`) -- **no. of datapoints $\times$ 1** : A column vector with each entry either 0 or 1 as entries.


* `feature_map`: (`QuantumCircuit` or `FeatureMap`) -- A quantum feature map which is the output of `feature_map()` defined earlier.


* `variational_form`: (`QuantumCircuit` or `VariationalForm`) -- A variational form which is the output of `variational_circuit()` defined earlier.


* `optimal_params`: (`numpy.ndarray`) -- the optimal parameters obtained after running the VQC algorithm above. These are the values obtained when the function `return_optimal_params()` is run. 


* `find_circuit_cost` : (`bool`) -- Calculates the circuit cost if set to `True`. Circuit cost is calculated by converting the circuit to the basis gate set `\[ 'u3', 'cx'\]` and then applying the formula **cost = 1$\times$(no.of u3 gates) + 10$\times$(no.of cx gates)**.


* `verbose` : (`bool`) -- prints the result message if set to `True`.

And gives as **output**: 

* `model_accuracy` : (`numpy.float64`) -- percent accuracy of the model. 


* `circuit_cost`: (`int`) -- circuit cost as explained above.


* `ans`: (`tuple`) -- Output of the `VQC.predict()` method. 


* `result_msg`: (`str`) -- Result message which also outputs the error message in case of one.


* `unrolled_circuit`: (`QuantumCircuit` or `None`) -- the circuit obtained after unrolling the full VQC circuit and substituting the optimal parameters to the basis gate set `\[ 'u3', 'cx'\]`.

**Note:** if you look inside the `grade()` function in Section 2 you'll see that we have initialized a COBYLA optimizer though the prediction step will not require one. Similarily we have given a dataset to `training dataset`. Both of these are dummy variables. The reason for this is because these are not optional variables the `VQC` class instantiation.  

In [113]:
def grade(test_data, test_labels, feature_map, variational_form, optimal_params, find_circuit_cost=True, verbose=True):
    seed = 10598
    model_accuracy = None 
    circuit_cost=None 
    ans = None
    unrolled_circuit = None
    result_msg=''
    data_dim = np.array(test_data).shape[1]
    dataset_size = np.array(test_data).shape[0]
    dummy_training_dataset=training_input = {'A':np.ones((2,data_dim)), 'B':np.ones((2, data_dim))}
    
    # converting 4's to 0's and 9's to 1's for checking 
    test_labels_transformed = np.where(test_labels==4, 0., 1.)
    max_qubit_count = 6
    max_circuit_cost = 2000
    
    # Section 1
    if feature_map is None:
        result_msg += 'feature_map variable is None. Please submit a valid entry' if verbose else ''
    elif variational_form is None: 
        result_msg += 'variational_form variable is None. Please submit a valid entry' if verbose else ''
    elif optimal_params is None: 
        result_msg += 'optimal_params variable is None. Please submit a valid entry' if verbose else ''
    elif test_data is None: 
        result_msg += 'test_data variable is None. Please submit a valid entry' if verbose else ''
    elif test_labels is None: 
        result_msg += 'test_labels variable is None. Please submit a valid entry' if verbose else ''
    elif not isinstance(feature_map, (QuantumCircuit, FeatureMap)):
        result_msg += 'feature_map variable should be a QuantumCircuit or a FeatureMap not (%s)' % \
                      type(feature_map) if verbose else ''
    elif not isinstance(variational_form, (QuantumCircuit, VariationalForm)):
        result_msg += 'variational_form variable should be a QuantumCircuit or a VariationalForm not (%s)' % \
                      type(variational_form) if verbose else ''
    elif not isinstance(test_data, np.ndarray):
        result_msg += 'test_data variable should be a numpy.ndarray not (%s)' % \
                      type(test_data) if verbose else ''
    elif not isinstance(test_labels, np.ndarray):
        result_msg += 'test_labels variable should be a numpy.ndarray not (%s)' % \
                      type(test_labels) if verbose else ''
    elif not isinstance(optimal_params, np.ndarray):
        result_msg += 'optimal_params variable should be a numpy.ndarray not (%s)' % \
                      type(optimal_params) if verbose else ''
    elif not dataset_size == test_labels_transformed.shape[0]:
        result_msg += 'Dataset size and label array size must be equal'
    # Section 2
    else:
        
        # setting up COBYLA optimizer as a dummy optimizer
        from qiskit.aqua.components.optimizers import COBYLA
        dummy_optimizer = COBYLA()

        # setting up the backend and creating a quantum instance
        backend = Aer.get_backend('qasm_simulator')
        backend_options = {"method": "statevector"}
        quantum_instance = QuantumInstance(backend, 
                                           shots=2000, 
                                           seed_simulator=seed, 
                                           seed_transpiler=seed, 
                                           backend_options=backend_options)

        # creating a VQC instance and running the VQC.predict method to get the accuracy of the model 
        vqc = VQC(optimizer=dummy_optimizer, 
                  feature_map=feature_map, 
                  var_form=variational_form, 
                  training_dataset=dummy_training_dataset)
        
        from qiskit.transpiler import PassManager
        from qiskit.transpiler.passes import Unroller
        pass_ = Unroller(['u3', 'cx'])
        pm = PassManager(pass_)
        # construct circuit with first datapoint
        circuit = vqc.construct_circuit(data[0], optimal_params)
        unrolled_circuit = pm.run(circuit)
        gates = unrolled_circuit.count_ops()
        if 'u3' in gates: 
            circuit_cost = gates['u3']
        if 'cx' in gates: 
            circuit_cost+= 10*gates['cx']
        
        if circuit.num_qubits > max_qubit_count:
            result_msg += 'Your quantum circuit is using more than 6 qubits. Reduce the number of qubits used and try again.'
        elif circuit_cost > max_circuit_cost:
            result_msg += 'The cost of your circuit is exceeding the maximum accpetable cost of 2000. Reduce the circuit cost and try again.'
        else: 
            
            ans = vqc.predict(test_data, quantum_instance=quantum_instance, params=np.array(optimal_params))
            model_accuracy = np.sum(np.equal(test_labels_transformed, ans[1]))/len(ans[1])

            result_msg += 'Accuracy of the model is {}'.format(model_accuracy) if verbose else ''
            result_msg += ' and circuit cost is {}'.format(circuit_cost) if verbose else ''
            
    return model_accuracy, circuit_cost, ans, result_msg, unrolled_circuit

## Process of grading using a dummy grading dataset

Let us create a dummy grading dataset with features and labels `grading_features` and `grading_labels` created from the last 2000 datapoints from `data_features` and `data_labels`so that we can a rough estimate of our accuaracy. It must be noted that this may not be a balanced dataset, i.e, may not have equal number of `4`'s and `9`'s and is not best practice. This is only given for the purpose of the demo of `grade()` function. In the final scoring done on HackerEarth, the testing dataset used will have a balanced number of class labels `4` and `9`.

In [114]:
grading_dataset_size=6000    # this value is not per digit but in total
grading_features = data_features[-grading_dataset_size:]
grading_labels = data_labels[-grading_dataset_size:]

In [115]:
start = time.process_time()

accuracy, circuit_cost, ans, result_msg, full_circuit  =  grade(test_data=grading_features, 
                                                                test_labels=grading_labels, 
                                                                feature_map=feature_map(), 
                                                                variational_form=variational_circuit(), 
                                                                optimal_params=return_optimal_params())

print("time taken: {} seconds".format(time.process_time() - start))
print(result_msg)

time taken: 140.37937009899997 seconds
Accuracy of the model is 0.7385 and circuit cost is 285


You can also check your **accuracy**, **circuit_cost** and **full_circuit** which is the result of combining the feature map and variational circuit and unrolling into the basis \['u3', 'cx'\].

In [116]:
print("Accuracy of the model: {}".format(accuracy))
print("Circuit Cost: {}".format(circuit_cost))
print("The complete unrolled circuit: ")
full_circuit.draw()

Accuracy of the model: 0.7385
Circuit Cost: 285
The complete unrolled circuit: 


┌───────────────┐  ┌───────────┐                                     »
q_0: ┤ U3(pi/2,0,pi) ├──┤ U3(0,0,8) ├─────■───────────────────────■────■──»
     ├───────────────┤┌─┴───────────┴──┐┌─┴─┐┌─────────────────┐┌─┴─┐  │  »
q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.6456) ├┤ X ├┤ U3(0,0,-3.9809) ├┤ X ├──┼──»
     ├───────────────┤├────────────────┤└───┘└─────────────────┘└───┘┌─┴─┐»
q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.8033) ├─────────────────────────────┤ X ├»
     └───────────────┘└────────────────┘                             └───┘»
c_0: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_1: ═════════════════════════════════════════════════════════════════════»
                                                                          »
c_2: ═════════════════════════════════════════════════════════════════════»
                                                                          »
«                             ┌───────────────┐  ┌───────────┐        »
«q_0: ─────────────────────■──┤ U3(pi/2,0,pi) ├──┤ U3(0,0,8) ├────────»
«                          │  └───────────────┘  └───────────┘        »
«q_1: ─────────────────────┼──────────■────────────────────────────■──»
«     ┌─────────────────┐┌─┴─┐      ┌─┴─┐      ┌────────────────┐┌─┴─┐»
«q_2: ┤ U3(0,0,-3.8456) ├┤ X ├──────┤ X ├──────┤ U3(0,0,10.388) ├┤ X ├»
«     └─────────────────┘└───┘      └───┘      └────────────────┘└───┘»
«c_0: ════════════════════════════════════════════════════════════════»
«                                                                     »
«c_1: ════════════════════════════════════════════════════════════════»
«                                                                     »
«c_2: ════════════════════════════════════════════════════════════════»
«                                                                     »
«                                                                          »
«q_0: ─────────────────────────────────────■───────────────────────■────■──»
«     ┌───────────────┐┌────────────────┐┌─┴─┐┌─────────────────┐┌─┴─┐  │  »
«q_1: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.6456) ├┤ X ├┤ U3(0,0,-3.9809) ├┤ X ├──┼──»
«     ├───────────────┤├────────────────┤└───┘└─────────────────┘└───┘┌─┴─┐»
«q_2: ┤ U3(pi/2,0,pi) ├┤ U3(0,0,1.8033) ├─────────────────────────────┤ X ├»
«     └───────────────┘└────────────────┘                             └───┘»
«c_0: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_1: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«c_2: ═════════════════════════════════════════════════════════════════════»
«                                                                          »
«                             ┌───────────────┐  ┌───────────┐        »
«q_0: ─────────────────────■──┤ U3(pi/2,0,pi) ├──┤ U3(0,0,8) ├────────»
«                          │  └───────────────┘  └───────────┘        »
«q_1: ─────────────────────┼──────────■────────────────────────────■──»
«     ┌─────────────────┐┌─┴─┐      ┌─┴─┐      ┌────────────────┐┌─┴─┐»
«q_2: ┤ U3(0,0,-3.8456) ├┤ X ├──────┤ X ├──────┤ U3(0,0,10.388) ├┤ X ├»
«     └─────────────────┘└───┘      └───┘      └────────────────┘└───┘»
«c_0: ════════════════════════════════════════════════════════════════»
«                                                                     »
«c_1: ════════════════════════════════════════════════════════════════»
«                                                                     »
«c_2: ════════════════════════════════════════════════════════════════»
«                                                                     »
«                                                                          »
«q_0: ─────────────────────────────────────■───────────────────────